In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
# Load the data
df = pd.read_excel('cleandata.xlsx')

# Drop the unnecessary columns
df = df.drop(['Kode Paket', 'Tahun Anggaran', 'Nama Paket', 'Tanggal Pengumuman', 'LPSE', 'Nama Pemenang', 'Kontrak'], axis=1)

In [3]:
# Fill in missing values for numerical and categorical columns
for column in df.columns:
    if df[column].dtype == np.number:
        df[column] = df[column].fillna(df[column].median())
    else:
        df[column] = df[column].fillna(df[column].mode()[0])

C:\Users\LENOVO\tf220\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\LENOVO\tf220\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\LENOVO\tf220\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
# function mengecek nilai missing value
def cek_null(df):
    col_na = df.isnull().sum().sort_values(ascending=True)
    percent = col_na / len(df)
    
    missing_data = pd.concat([col_na, percent], axis=1, keys=['Total', 'Percent'])
    
    if (missing_data[missing_data['Total'] > 0].shape[0] == 0):
        print("Tidak ditemukan missing value pada dataset")
        
    else:
        print(missing_data[missing_data['Total'] > 0])

In [5]:
cek_null(df)

Tidak ditemukan missing value pada dataset


In [6]:
# Perform label encoding for categorical variables
le = LabelEncoder()
categorical_columns = ['Kategori', 'Sumber Dana', 'Nama KLPD', 'Satker']
for column in categorical_columns:
    df[column] = le.fit_transform(df[column])

In [7]:
# Split the dataset into features (X) and target (y)
X = df.drop('Skor Total', axis=1)
y = df['Skor Total']

# Split the dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# Normalize numerical features
scaler = MinMaxScaler()
numerical_columns = ['HPS', 'PAGU']
X_train[numerical_columns] = scaler.fit_transform(X_train[numerical_columns])
X_test[numerical_columns] = scaler.transform(X_test[numerical_columns])

In [9]:
# Define the model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=[X_train.shape[1]]),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [10]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'mse'])

In [11]:
# Train the model
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

Epoch 1/50
2146/2146 [==============================] - 2s 1ms/step - loss: 1222.7953 - mae: 14.9570 - mse: 1222.7953 - val_loss: 664.4351 - val_mae: 21.6963 - val_mse: 664.4351
Epoch 2/50
2146/2146 [==============================] - 2s 1ms/step - loss: 254.3930 - mae: 12.5083 - mse: 254.3930 - val_loss: 172.7717 - val_mae: 10.4331 - val_mse: 172.7717
Epoch 3/50
2146/2146 [==============================] - 2s 935us/step - loss: 257.0757 - mae: 12.5909 - mse: 257.0757 - val_loss: 307.4518 - val_mae: 14.2461 - val_mse: 307.4518
Epoch 4/50
2146/2146 [==============================] - 2s 973us/step - loss: 224.4687 - mae: 11.8118 - mse: 224.4687 - val_loss: 185.2652 - val_mae: 10.8497 - val_mse: 185.2652
Epoch 5/50
2146/2146 [==============================] - 2s 1ms/step - loss: 204.7471 - mae: 11.3541 - mse: 204.7471 - val_loss: 177.5476 - val_mae: 10.6102 - val_mse: 177.5476
Epoch 6/50
2146/2146 [==============================] - 2s 1ms/step - loss: 185.2573 - mae: 10.8342 - mse: 185.257

2146/2146 [==============================] - 2s 865us/step - loss: 130.7789 - mae: 9.1392 - mse: 130.7789 - val_loss: 131.6743 - val_mae: 9.1526 - val_mse: 131.6743
Epoch 48/50
2146/2146 [==============================] - 2s 865us/step - loss: 130.7707 - mae: 9.1371 - mse: 130.7707 - val_loss: 132.3409 - val_mae: 9.1701 - val_mse: 132.3409
Epoch 49/50
2146/2146 [==============================] - 2s 875us/step - loss: 130.7595 - mae: 9.1342 - mse: 130.7595 - val_loss: 133.0806 - val_mae: 9.2077 - val_mse: 133.0806
Epoch 50/50
2146/2146 [==============================] - 2s 870us/step - loss: 130.8739 - mae: 9.1382 - mse: 130.8739 - val_loss: 132.3153 - val_mae: 9.1579 - val_mse: 132.3153


In [12]:
# Evaluate the model on the test set
model.evaluate(X_test, y_test)

537/537 [==============================] - 0s 551us/step - loss: 132.3153 - mae: 9.1579 - mse: 132.3153


[132.3152618408203, 9.157934188842773, 132.3152618408203]

In [13]:
model.save("model_v2.h5")

In [14]:
# Save the label encoder and the scaler
joblib.dump(le, 'label_encoder.joblib')
joblib.dump(scaler, 'scaler.joblib')

NameError: name 'joblib' is not defined